In [1]:
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import umap
import json
import hdbscan
import numpy as np
from collections import defaultdict

In [2]:
init_notebook_mode(connected=True)

In [3]:
filename="atari"

In [4]:
data = []
with open(f"./output/{filename}.jsonl") as file:
    for line in file:
        data.append(json.loads(line))
%time

CPU times: user 2 µs, sys: 1 µs, total: 3 µs
Wall time: 3.81 µs


In [5]:
data_features = []
labels = []
text = []
for sentence in data:
    word = sentence["linex_index"].split("-")[0]
    features = sentence['features']
    for feature in features:
        if feature["token"] == word: #possible error from change due to tokenization
            labels.append(word)
            data_features.append(feature["layers"][0]["values"])
            text.append(" ".join([feature["token"] for feature in sentence["features"]]))
#             break # only one context per sentence

In [6]:
first_label = labels[0]
for PARTITION in range(len(labels)):
    if labels[PARTITION] != first_label:
        break

In [7]:
reducer = umap.UMAP(n_neighbors=80, min_dist=0, init='random', n_epochs=30)#, metric="cosine")

In [8]:
reducer2 = umap.UMAP(n_neighbors=30, min_dist=0, init='random', n_epochs=11)#, metric="cosine")

In [11]:
embedding = reducer.fit_transform(data_features[:PARTITION])
embedding2 = reducer.fit_transform(data_features[PARTITION:])

In [8]:
embedding3 = reducer.fit_transform(data_features)

In [13]:
clusters = hdbscan.HDBSCAN().fit_predict(embedding)
clustered = (clusters >= 0)

clusters2 = hdbscan.HDBSCAN().fit_predict(embedding2)
clustered2 = (clusters2 >= 0)

ValueError: k must be less than or equal to the number of training points

In [9]:
clusters3 = hdbscan.HDBSCAN().fit_predict(embedding3)
clustered3 = (clusters3 >= 0)

In [10]:
fig = go.Figure(
    data=go.Scatter(
        x=embedding[clustered, 0],
        y=embedding[clustered, 1],
        hovertext=(np.array(text)[:PARTITION])[clustered],
        mode='markers',
        marker=dict(
            color=clusters[clustered],#[word_to_id[label] for label in labels[:500]],
            line=dict(
                width=1,
                color='DarkSlateGrey'
            ),
        )
    )
)

fig2 = go.Figure(
    data=go.Scatter(
        x=embedding2[clustered2, 0],
        y=embedding2[clustered2, 1],
        hovertext=(np.array(text)[PARTITION:])[clustered2],
        mode='markers',
        marker=dict(
            color=clusters2[clustered2], #[word_to_id[label] for label in labels[500:]],
            line=dict(
                width=1,
                color='DarkSlateGrey'
            ),
        )
    )
)

NameError: name 'embedding' is not defined

In [10]:
fig3 = go.Figure(
    data=go.Scatter(
        x=embedding3[clustered3, 0],
        y=embedding3[clustered3, 1],
        hovertext= np.array(text)[clustered3],
        mode='markers',
        marker=dict(
            color=clusters3[clustered3],#[word_to_id[label] for label in labels[:500]],
            line=dict(
                width=1,
                color=[0 if ind < PARTITION else 1 for (ind, _) in enumerate(labels) if clustered3[ind]]#'DarkSlateGrey'
            ),
        )
    )
)

In [15]:
fig.show()

In [16]:
fig2.show()

In [11]:
fig3.show()

In [12]:
plot(fig3, filename=f"./output/graphs/{filename}.html")

'./output/graphs/side.html'

In [19]:
plot(fig, filename=f"./output/graphs/state.html")

'./output/graphs/state.html'